<a href="https://colab.research.google.com/github/ParkSoeon/NLP_team1/blob/main/team1_PsycholCounsChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

※ 실험 중

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import pandas as pd
from datasets import Dataset, load_dataset

df = pd.read_csv('/content/drive/MyDrive/[BITAmin24S]Chatbot/merged_df.csv')
df

,context,response
0,"저는 제 감정과 저 자신과 관련된 문제를 겪고 있습니다. 거의 잠을 자지 않고, 내...","만약 모든 사람들이 당신을 무가치하다고 생각한다면, 새로운 사람들을 찾아 사귀어야 ..."
1,"저는 제 감정과 저 자신과 관련된 문제를 겪고 있습니다. 거의 잠을 자지 않고, 내...","안녕하세요, 질문해 주셔서 감사합니다. 자신이 무가치하다고 느끼는 것은 불행히도 흔..."
2,"저는 제 감정과 저 자신과 관련된 문제를 겪고 있습니다. 거의 잠을 자지 않고, 내...",제가 가장 먼저 제안하는 것은 필요한 수면을 충분히 취하는 것입니다. 그렇지 않으면...
3,"저는 제 감정과 저 자신과 관련된 문제를 겪고 있습니다. 거의 잠을 자지 않고, 내...",치료는 우울하고 자신을 무가치하게 느끼는 사람들에게 필수적입니다. 우울감과 자존감 ...
4,"저는 제 감정과 저 자신과 관련된 문제를 겪고 있습니다. 거의 잠을 자지 않고, 내...",먼저 당신이 느끼는 감정이 혼자가 아니라는 것을 알아주셨으면 합니다. 항상 도와줄 ...
...,...,...
8053,내 손자의 계모는 벌로써 그를 핑크색 바비 배낭을 메고 학교에 보내고 있다.,절대 아닙니다! 굴욕을 처벌로 사용하는 것은 절대 아이의 이익에 부합하지 않습니다....
8054,내 남자친구는 마약 중독에서 회복 중입니다. 최근에 싸움을 했고 그는 매우 거리를 ...,당신과 남자친구 사이에 긴장이 있어서 유감입니다. 관계는 두 사람이 서로 연결되는 ...
8055,출생모는 임신 중 여러 번 자살을 시도했습니다. 양모는 아기를 입양할 때 시한부 판...,"진정한 대답은 ""확실히 말할 수 있는 사람은 아무도 없다""는 것입니다. 변수는 이 ..."
8056,"저는 성인 생활이 그를 우울하게 만들고 있으며, 우리는 주말에 종종 오후 1시나 2...",그가 제공하는 것보다 더 많은 것이 필요하다고 자신을 어떻게 설득하나요? 이 관계에...


In [ ]:
!pip install transformers datasets

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# CSV 파일 로드
df = pd.read_csv('/content/drive/MyDrive/[BITAmin24S]Chatbot/merged_df.csv')

# 데이터 전처리 함수 정의
def preprocess_data(row):
    return f"Context: {row['context']}\nResponse: {row['response']}"

df['text'] = df.apply(preprocess_data, axis=1)

# Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df[['text']])

# 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>')
import os
from datasets import Dataset, load_dataset
from transformers import PreTrainedTokenizerFast, AutoModelForCausalLM, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel

# 적절한 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>')
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

# eos_token을 pad_token으로 설정
tokenizer.pad_token = tokenizer.eos_token
# 또는 새로운 pad_token 추가
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# model.resize_token_embeddings(len(tokenizer))

# 데이터셋 토크나이징
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# DataCollator 설정
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Map:   0%|          | 0/8058 [00:00<?, ? examples/s]

In [ ]:
import os
from datasets import Dataset, load_dataset
from transformers import PreTrainedTokenizerFast, AutoModelForCausalLM, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel

# 적절한 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>')
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
# Split the dataset into train and eval
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [ ]:
tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map:   0%|          | 0/8058 [00:00<?, ? examples/s]

In [ ]:
 from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./kogpt2-therapy-chatbot",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    evaluation_strategy="no",  # Disable evaluation
    load_best_model_at_end=False,  # Disable loading the best model
    weight_decay=0.01,
    learning_rate=5e-5,
    warmup_steps=1_000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,  # Use the tokenized training dataset
    data_collator=data_collator,
    # Remove the callbacks if there's no evaluation
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# 모델 학습
trainer.train()

Step,Training Loss
500,0.768600
1000,0.812300
1500,0.868100
2000,0.796800
2500,0.581600
3000,0.568800
3500,0.502500
4000,0.466400
4500,0.314200
5000,0.292900


TrainOutput(global_step=6045, training_loss=0.5441224752131132, metrics={'train_runtime': 674.6745, 'train_samples_per_second': 35.831, 'train_steps_per_second': 8.96, 'total_flos': 1579118395392000.0, 'train_loss': 0.5441224752131132, 'epoch': 3.0})

In [ ]:
 def chatbot_response(context):
    # 프롬프트에 전문적인 지시와 역할 설정 추가
    input_text = (
        f"User: {context}\n\n"
        "AI: 당신은 전문적인 정신의학 상담가입니다. 내담자의 아픔에 공감할 수 있는 공감적인 말하기도 잘 하는 상담자입니다. 내담자의 고민에 대해 답변을 하되 max_length=300이라는 것을 유의하여 말이 끊기지 않게 유의하십시오."
    )
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to('cuda')  # GPU로 이동

    # Generate response with adjusted parameters
    output = model.generate(
        input_ids,
        max_length=300,  # 충분히 긴 응답을 위해 max_length를 300으로 설정
        pad_token_id=tokenizer.eos_token_id,
        top_p=0.85,  # Nucleus sampling
        top_k=100,   # Top-k sampling
        temperature=0.6,  # 랜덤성 조절
        repetition_penalty=1.3,  # 반복 어구 벌점
        no_repeat_ngram_size=2   # 2-gram 반복 방지
    )

    # Decode and process the output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the part after "AI:" and remove unwanted context
    response_start = generated_text.find("AI:") + len("AI:")
    response = generated_text[response_start:].strip()

    # Ensure the context is removed if present
    if response.startswith(context):
        response = response[len(context):].strip()

    # 문장의 자연스러운 끊김을 위해 후처리
    if len(response) > 0 and response[-1] not in [".", "!", "?"]:
        response += "."

    return response

print(chatbot_response("요즘 너무 불안해요."))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:562: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `100` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos to

당신은 전문적인 정신의학 상담가입니다. 내담자의 아픔에 공감할 수 있는 공감적인 말하기도 잘 하는 상담자입니다. 내담자의 고민에 대해 답변을 하되 max_length=300이라는 것을 유의하여 말이 끊기지 않게 유의하십시오. 가족 주치의와 약속을 잡고 당신의 걱정에 대해 논의해 보세요. 때로는 우리가 경험을 놓칠까 봐 두려워해서 그걸 해야 할 때도 있습니다. 치료 세션 동안 무엇이 당신을 괴롭히고 있는지 이해하도록 노력해보세요 --당신이 고통스러운 감정을 느끼고 있다는 걸 알면서도 왜 그런지 설명해줄 거예요. 하지만 항상 그렇게 느껴지는 것은 아닙니다. 변화를 준비하고 행동하는 것도 중요합니다. 변화는 종종 건강하고 긍정적인 경험이어야 합니다. "아니요," 또는 "절대"라는 용어를 사용하지 마세요(동물이 아니라 말을 듣는 것입니까?) 동물은 마치 누군가가 대신 말하고 상처받을 것처럼 느껴질 수도 있습니다(“반드시려 한다"는 느낌입니다). 이 두려움을 이해하고 자신을 다른 전제가 가능한 방식으로 바꾸어 나오도록 연습하세요.
Response: 안녕하세요, 질문해주셔서 감사합니다, 매우 중요한 문제입니다
! 이는 좀 더 어렵고 많은 정보를 알지 못한 채 조언을 해주실 필요는 없습니다. 그러나 추가적인 정보 없이 상담을 받을 수는 없지만, 몇 가지 도움이 될 수 있을 것입니다. 전문가와 이야기하기 시작점이나 열린 대화를 나누는 것이 유익이 되기를 바랍니다. 여러분의 이야기를 들을.


In [ ]:

# Repository 생성 & model upload
REPO_NAME = 'YOUR_REPO_NAME_TO_MAKE' # ex) 'my-bert-fine-tuned'
AUTH_TOKEN = 'YOUR_TOEKN' # <https://huggingface.co/settings/token>

## Upload to Huggingface Hub
model.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    use_auth_token=AUTH_TOKEN
)
tokenizer.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    use_auth_token=AUTH_TOKEN
)


HfHubHTTPError:  (Request ID: Root=1-66c87b55-432d88c2171a18867be096a8;0e88a6be-7fc2-4da4-9e55-a579c0bdc1bc)

403 Forbidden: You don't have the rights to create a model under the namespace "ParkSoeon".
Cannot access content at: https://huggingface.co/api/repos/create.
If you are trying to create or update content,make sure you have a token with the `write` role.